In [1]:
import pandas as pd

electorates = pd.read_csv("https://www.elections.act.gov.au/__data/assets/file/0014/1002641/Electorates.txt")
candidates = pd.read_csv("https://www.elections.act.gov.au/__data/assets/text_file/0018/1002636/Candidates.txt")
groups = pd.read_csv("https://www.elections.act.gov.au/__data/assets/file/0017/1002644/Groups.txt")

In [2]:
murrumbidgee = candidates[candidates["ecode"] == 4]
groups[(groups["ecode"] == 4) & (groups["pcode"] == 0)]["pname"]

26    LIKE CANBERRA
Name: pname, dtype: object

In [3]:
electorates

,ecode,electorate
0,1,Brindabella
1,2,Ginninderra
2,3,Kurrajong
3,4,Murrumbidgee
4,5,Yerrabi


In [5]:
candidates

,ecode,pcode,ccode,cname
0,1,0,0,"HASTIR, Claude"
1,1,0,1,"KEMP, Melissa"
2,1,1,0,"FAZEY-SOUTHWELL, Annette"
3,1,1,1,"PARTON, Mark"
4,1,1,2,"COCKS, Ed"
...,...,...,...,...
136,5,6,0,"DEWSON, Andrew"
137,5,6,1,"KENNETT, Susie"
138,5,7,0,"POLLARD, David"
139,5,7,1,"COTTINGHAM, Mandy"


In [6]:
groups

,ecode,pcode,pname,pabbrev,cands
0,1,0,SUSTAINABLE AUSTRALIA (ACT),SUST,2
1,1,1,CANBERRA LIBERALS,LIB,5
2,1,2,ANIMAL JUSTICE PARTY,AJP,2
3,1,3,LIKE CANBERRA,LIKE,2
4,1,4,ACT LABOR,ALP,5
5,1,5,LIBERAL DEMOCRATS,LDP,5
6,1,6,THE GREENS,GREEN,3
7,1,7,AUSTRALIAN SEX PARTY ACT,ASP,2
8,1,8,UNGROUPED,UNG,2
9,2,0,SUSTAINABLE AUSTRALIA (ACT),SUST,2


In [137]:
def get_party(first_name, last_name):
    candidate = candidates[candidates["cname"].apply(lambda x: (first_name in x) and (last_name in x))]
    ecode = candidate.loc[candidate.index[0]]["ecode"]
    pcode = candidate.loc[candidate.index[0]]["pcode"]
    party = groups[(groups["pcode"] == pcode) & (groups["ecode"] == ecode)]
    return party.loc[party.index[0]]["pname"]

final_data = pd.DataFrame(columns=["electorate", "party", "progress"])

electorates = ["Brindabella", "Ginninderra", "Kurrajong", "Murrumbidgee", "Yerrabi"]
for electorate in electorates:
    results = pd.read_excel("table2." + electorate + "22Oct.xlsx", header=2)
    results = results[results.columns[1:-3]]
    results.columns = list(results.columns)[:-1] + ["exhausted"]
    results["step"] = list(range(len(results)))
    # bri_results.sort_values("step", inplace=True)
    # bri_results[bri_results["step"] == bri_results["step"]]
    results = results[(results["step"] == 0) | (results["step"] % 2 != 0)]
    results = results.T
    results = results.iloc[:-1]
    progress = []
    parties = []
    for index, row in results.iterrows():
        steps = []
        for column in results.columns:
            steps.append(row[column])
        if index == "exhausted":
            party = "nan"
        else:        
            first_name = index.split()[0]
            last_name = index.split()[1]
            party = get_party(first_name, last_name)        
        parties.append(party)        
        progress.append(steps)
    results["progress"] = progress
    results["party"] = parties
    results["electorate"] = [electorate] * len(results)
    final_data = pd.concat([final_data, results[["electorate", "progress", "party"]]])
final_data.to_csv("finalVotes.csv", index_label="candidate")